## 6.5 The GMRES Method

In [ ]:
import numpy as np
import time

**Implemention 6.27: GMRES with modified Gram-Schmidt**

We implement the GMRES method, using `numpy` functions for scalar products and norms.

In [ ]:
def gmres(A, b, x, m, eps):
    n = A.shape[0]
    x = x.copy()
    d = b.copy() - A.dot(x)
    b = np.zeros(m + 1)
    b[0] = np.linalg.norm(d)
    
    v = np.zeros((n, m + 1))
    h = np.zeros((m + 1, m))
    R = np.zeros((m, 2, 2))
    
    v[:, 0] = d / b[0]

    for k in range(m):
        w = A.dot(v[:, k])
        for j in range(k + 1):
            h[j, k] = np.dot(w, v[:, j])
            w -= h[j, k] * v[:, j]
        h[k + 1, k] = np.linalg.norm(w)
        
        if np.abs(h[k + 1, k]) > 1e-12:
            v[:, k + 1] = w / h[k + 1, k]
            
            for j in range(k):
                h[j:j + 2, k] = R[j].dot(h[j:j + 2, k])

            alpha = np.linalg.norm(h[k:k + 2, k])

            R[k, 0, 0] = h[k, k] / alpha
            R[k, 0, 1] = h[k + 1, k] / alpha
            R[k, 1, 1], R[k, 1, 0] = R[k, 0, 0], -R[k, 0, 1]
            
            h[k:k + 2, k] = alpha, 0
            b[k + 1] = R[k, 1, 0] * b[k]
            b[k] = R[k, 0, 0] * b[k]
            if np.abs(b[k + 1]) < eps:
                break
        else:
            break
    
    y = np.zeros(k + 1)
    y[-1] = b[-1] / h[k, k]
    for i in range(k - 2, -1, -1):
        y[i] = (b[i] - h[i, i + 1:k + 1].dot(y[i + 1:])) / h[i, i] 
    for i in range(k):
        x += y[i] * v[:, i]
    return x, k

#### Example 6.28

We consider the model matrix and apply our implementation of the GMRES method. This yields the following.

In [ ]:
for m in range(20, 101, 10):
    n = m**2
    N = np.diag(np.ones(m - 1), 1) + np.diag(np.ones(m - 1), -1)
    B = 4 * np.eye(m) - N
    A = np.kron(np.eye(m), B) - np.kron(N, np.eye(m))
    b = np.ones(n)
    x0 = np.zeros(n)

    t = time.perf_counter()
    xgmres, k = gmres(A, b, x0, n, eps=1e-6)
    t = time.perf_counter() - t

    res = np.linalg.norm(b - np.dot(A, xgmres))
    xexact = np.linalg.solve(A, b)
    print(f'GMRES, n={n:05d}, Steps={k:03d}, Time={t:07.4f}sec, res={res:4.2e}, Error={np.linalg.norm(xgmres - xexact):4.2e}')

We see that the number of needed steps is comparable to the number needed by the CG method. The GMRES is more expensive, however, it can be applied to general linear systems.